In [2]:
import re
import math
import time
import numpy
import requests
import contextlib
from bs4 import BeautifulSoup

In [72]:
### 存取設定範圍內之電影連結

# Release Date
r_date_min = '2015-01-01'
r_date_max = '2016-03-31'

# User Rating
u_rating_min = 5.0 
u_rating_max = 10 

# Runtime
rt = 60

# Title Type
movie_feature = 'feature'

url = 'http://www.imdb.com/search/title?release_date='+r_date_min+','+r_date_max+'&runtime='+str(rt)+',&title_type='+movie_feature +'&user_rating='+str(u_rating_min)+','+str(u_rating_max)
res = requests.get(url, verify=True)
sp = BeautifulSoup(res.text)
print url

total = ''.join(sp.find('div', {'id':'left'}).text.split('of')[1].split('\n')[0].split(','))
print 'Movies:  ' + str(total)

k=0
tStart = time.time()

with open('data/IMDB/movies_link.txt', 'w') as f1:
    
    for i in range(0,(int(total)/50)+1):
        
        try:
            each_url = url + '&start=' +str(i*50+1)
            #print each_url
            
            res = requests.get(each_url, verify=True)
            res.encoding = 'utf-8'
            soup = BeautifulSoup(res.text)
            title = soup.findAll('td', {'class':'title'})
            
            for a in title:
                f1.write('http://www.imdb.com' + a.find('a')['href'])
                f1.write('\n')
                
        except BaseException, e:
            print e

with open('data/IMDB/movies_link.txt', 'r') as f2: 

    for j in f2.readlines():
        k=k+1
        
print 'Write In: ' + str(k)
print 'Duration: ' + time.strftime("%H:%M:%S", time.gmtime(time.time() - tStart))

http://www.imdb.com/search/title?release_date=2015-01-01,2016-03-31&runtime=60,&title_type=feature&user_rating=5.0,10
Movies:   3164
Write In: 3164
Duration: 00:02:37


In [73]:
with contextlib.nested(open('data/IMDB/movies_link.txt', 'r'),
                       open('data/IMDB/movies_detail.txt', 'w'),
                       open('data/IMDB/movies_director.txt', 'w'),
                       open('data/IMDB/movies_writer.txt', 'w'),
                       open('data/IMDB/movies_actor.txt', 'w'),
                       open('data/IMDB/movies_miss.txt', 'w')) as (f0,f1,f2,f3,f4,f5): 

    tStart = time.time()
    
    for url in f0.readlines():
        
        try:
            res0 = requests.get(url.strip(), verify=True)
            res0.encoding = 'utf-8'
            sp0 = BeautifulSoup(res0.text)

            # IMDB movie sno
            sno =  url.split('title/')[1].replace('/', '').strip()

            # IMDB movie title
            #movie_title = sp0.find('span', {'class':'itemprop'}).text
            #movie_title = sp0.find('div', {'id':'ratingWidget'}).find('strong').text
            try:
                movie_title = sp0.find('div', {'id':'ratingWidget'}).find('strong').text.strip()
            except AttributeError, ae:
                movie_title = re.sub(r'(\(\))', '', re.sub(r'(\d)', '', sp0.find('h1', {'itemprop':'name'}).text.strip()))
  
            # IMDB movie duration
            movie_duration = sp0.find('time', {'itemprop':'duration'})['datetime'].replace('PT','').replace('M','')
            
            # IMDB movie genre
            genre = []
            movie_story = sp0.find('div', {'id':'titleStoryLine'})
            item = movie_story.find('div', {'itemprop':'genre'})   
            try:
                for g in item.findAll('a'):
                    genre.append(g.text.strip())
                    
            except BaseException, e:
                genre.append('None')
                
            movie_genre = ','.join(genre)
            
            # IMDB movie datePublished
            movie_date = sp0.find('meta', {'itemprop':'datePublished'})['content']

            # IMDB movie rate value
            try:
                movie_rate_value = sp0.find('span', {'itemprop':'ratingValue'}).text
            except BaseException, e:
                movie_rate_value = ''
                
            # 123,456 IMDb users have given a weighted average vote of 9.5/10
            try:
                movie_rate_count = sp0.find('span', {'itemprop':'ratingCount'}).text.replace(',', '')
            except BaseException, e:
                movie_rate_count = ''
            
            # Write in detail.txt
            f1.write(sno + ";" + movie_title.encode('utf-8') + ";" + movie_duration.encode('utf-8') + ";" + movie_genre.encode('utf-8') + ";" + movie_date.encode('utf-8') + ";" + movie_rate_value.encode('utf-8') + ";" + movie_rate_count.encode('utf-8'))
            f1.write('\n')

            movie_fullcredits_url = url.strip() + 'fullcredits'
            res1 = requests.get(movie_fullcredits_url, verify=True)
            res1.encoding = 'utf-8'
            sp1 = BeautifulSoup(res1.text)

            # IMDB movie director/writer
            fullcredits = sp1.select("#fullcredits_content")[0]
            h4_class =  fullcredits.findAll('h4', {'class':'dataHeaderWithBorder'})
            table_class = fullcredits.findAll('table')
            
            if len(h4_class)>=3:
                k=3
            elif len(h4_class) == 0:
                k=0
            elif len(h4_class) == 1:
                k=1
            else:
                k=2
                
            for i in range(k):

                a = re.match(r'([a-z|A-Z]+)' '?.*', h4_class[i].text.strip())    

                if a.groups(1)[0] == 'Directed':
                    type4t = 'director'
                if a.groups(1)[0] == 'Writing':
                    type4t = 'writer'    
                if a.groups(1)[0] == 'Cast':
                    type4t = 'cast'

                tr_class = table_class[i].findAll("tr")           

                for td in tr_class:
                    for aa in td.findAll('a'):

                        # Search name Not character
                        if re.search(r'(name)', aa['href']):

                            # Write in actor.txt
                            if len(aa)!=1:
                                f4.write(sno + ';' + type4t + ';' + aa.text.encode('utf-8').strip())
                                f4.write('\n')
                            
                            # Write in director.txt
                            if a.groups(1)[0] == 'Directed':
                                f2.write(sno + ';' + type4t + ';' + aa.text.encode('utf-8').strip())
                                f2.write('\n')
                            
                            # Write in writer.txt
                            if a.groups(1)[0] == 'Writing':
                                f3.write(sno + ';' + type4t + ';' + aa.text.encode('utf-8').strip())
                                f3.write('\n')
                            
        except BaseException, e:
            print e
            print url.strip()
            f5.write(url.strip())
            f5.write('\n')            

with open('data/IMDB/movies_detail.txt', 'r') as f: 
    
    k=0
    for j in f.readlines():
        k=k+1
        
print 'Movies: ' + str(k)
print 'Duration: ' + time.strftime("%H:%M:%S", time.gmtime(time.time() - tStart))

('Connection aborted.', error(10054, '\xbb\xb7\xba\xdd\xa5D\xbe\xf7\xa4w\xb1j\xa8\xee\xc3\xf6\xb3\xac\xa4@\xad\xd3\xb2{\xa6s\xaa\xba\xb3s\xbdu\xa1C'))
http://www.imdb.com/title/tt4428720/
Movies: 3163
Duration: 02:19:19


In [70]:
### 單一部電影資料測試

url = 'http://www.imdb.com/title/tt5044906/'
res0 = requests.get(url, verify=True)
res0.encoding = 'utf-8'
sp0 = BeautifulSoup(res0.text)

# IMDB movie title
try:
    movie_title = sp0.find('div', {'id':'ratingWidget'}).find('strong').text
except AttributeError, ae:
    movie_title = re.sub(r'(\(\))', '', re.sub(r'(\d)', '', sp0.find('h1', {'itemprop':'name'}).text.strip()))
    
print movie_title

# IMDB movie duration
movie_duration = sp0.find('time', {'itemprop':'duration'})['datetime'].replace('PT','').replace('M','')
print movie_duration

# IMDB movie genre
genre = []
movie_story = sp0.find('div', {'id':'titleStoryLine'})
item = movie_story.find('div', {'itemprop':'genre'})
try:
    for g in item.findAll('a'):
        genre.append(g.text.strip())  
except BaseException, e:
    genre.append('None') 
print ','.join(genre)

# IMDB movie datePublished
movie_date = sp0.find('meta', {'itemprop':'datePublished'})
print movie_date['content']

# IMDB movie rate value
try:
    movie_rate_value = sp0.find('span', {'itemprop':'ratingValue'})
except BaseException, e:
    movie_rate_value = ''
print movie_rate_value.text

# xxx,xxx IMDb users have given a weighted average vote of 8.5/10
try:
    movie_rate_count = sp0.find('span', {'itemprop':'ratingCount'})
except BaseException,e:
    movie_rate_count = ''
print movie_rate_count.text

# IMDB movie fullcredits
# https://github.com/gtb01026/YB101/blob/master/PycharmProjects/BoxOffice/IMDBComplete/release_1970_2010/combine.py
movie_fullcredits_url = url + 'fullcredits'
res1 = requests.get(movie_fullcredits_url, verify=True)
res1.encoding = 'utf-8'
sp1 = BeautifulSoup(res1.text)

# IMDB movie director/writer
#movie_director = sp1.find('div', {'id':'fullcredits_content'})
fullcredits = sp1.select("#fullcredits_content")[0]
h4_class =  fullcredits.findAll('h4', {'class':'dataHeaderWithBorder'})
table_class = fullcredits.findAll('table')
print len(h4_class)

if len(h4_class)>=3:
    k=3
elif len(h4_class) == 0:
    k=0
elif len(h4_class) == 1:
    k=1
else:
    k=2

for i in range(k):
    #print repr(h4_class[i].text.strip())

    credits_list1 = []
    a = re.match(r'([a-z|A-Z]+)' '?.*', h4_class[i].text.strip())    

    if a.groups(1)[0] == 'Directed':
        type4t = 'director'
    if a.groups(1)[0] == 'Writing':
        type4t = 'writer'    
    if a.groups(1)[0] == 'Cast':
        type4t = 'cast'

    tr_class = table_class[i].findAll("tr")
        
    #print a.groups(1)[0]  

    print type4t

    for td in tr_class:
        for a in td.findAll('a'):    
            if re.search(r'(name)', a['href']):
                credits_list1.append(a.text.strip())
    print ','.join(credits_list1).replace(',,', ',')

Jan Quadrant Vincent 16
133
Action
2015-09-20
9.6
20
1
director
Rick Sanchez


In [3]:
with open('data/IMDB/movies_test.txt', 'w') as f: 
    url = 'http://www.imdb.com/title/tt4635548/'
    res0 = requests.get(url, verify=True)
    res0.encoding = 'utf-8'
    sp0 = BeautifulSoup(res0.text)

    # IMDB movie title
    movie_title = sp0.find('span', {'class':'itemprop'}).text
    print movie_title
    f.write(movie_title.encode('utf-8'))
f.close()

Wrong No.


In [45]:
m = re.sub(r'(\(\))', '', re.sub(r'(\d)', '', 'Driving While Black (2016)'))
print m

Driving While Black 
